In [ ]:
import pandas as pd
import plotly.express as px

fileparts = {
    200: "200mW",
    400: "400mW",
    600: "600mW",
    800: "800mW",
    1000: "1W",
    1200: "1,2W",
    1400: "1,4W",
    1600: "1,6W",
    1800: "1,8W",
    2000: "2W",
}

gate_names = ('C2H4', 'C2H3O', 'Parent')
gates = ((23, 30), (38, 46), (53, 59))


def gate(mq):
    for gate_name, (low, high) in zip(gate_names, gates):
        if low <= mq <= high:
            return gate_name
    return "Other"


full_df = pd.DataFrame()

for power, name in fileparts.items():
    df = pd.read_hdf(fr"J:\ctgroup\Edward\DATA\VMI\20250408\calibrated\po_{name}_lin_calibrated.h5", key="data")
    p_max = 0.6
    df_filtered = df.query(
            f'(px > -{p_max}) & (px < {p_max}) & (py > -{p_max}) & (py < {p_max}) & (pz > -{p_max}) & (pz < {p_max})')
    df_filtered = df_filtered[(df_filtered['m/q'] > 0) & (df_filtered['m/q'] < 100)]
    df_filtered['gate'] = df_filtered['m/q'].apply(gate)
    df_filtered.sort_values(by=['m/q'], ascending=False, inplace=True)
    df_filtered['power'] = power
    df_filtered['normalization'] = 1 / len(df_filtered)
    print(f"Power: {power} mW, Number of events: {len(df_filtered)}")
    full_df = pd.concat([full_df, df_filtered], ignore_index=True)
    print(f"Total number of events: {len(full_df)}, Total size: {full_df.memory_usage(deep=True).sum() / 1e6:.2f} MB")
    del df_filtered, df

full_df.head()

In [ ]:
fig = px.density_heatmap(
        full_df,
        x="power",
        y='m/q',
        z="normalization",
        marginal_y="histogram",
)
fig.show()